In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
df = pd.read_csv('../input/fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [6]:
# droping any null value
df = df.dropna()

In [7]:
# getting the independent features
X = df.drop('label',axis=1)

In [8]:
y = df['label']

In [9]:
print('shape of X',X.shape)

print('shape of y',y.shape)

shape of X (18285, 4)
shape of y (18285,)


In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.4.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
## vocabulary size
vocab_size = 5000

In [14]:
# one_hot representation
messages = X.copy()

In [15]:
messages.reset_index(inplace = True)

In [16]:
messages.title

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
#data preprocessing
ps = PorterStemmer()
lem = WordNetLemmatizer()

In [20]:
def clean_text(text):
  
        text = re.sub('[^a-zA-Z0-9]',' ', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
        text = text.lower()
        
         
        return text

In [22]:
def text_process(text):
    
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [23]:
messages.title = messages.title.apply(clean_text)

In [24]:
messages.title

0        house dem aide  we didn t even see comey s let...
1        flynn  hillary clinton  big woman on campus   ...
2                        why the truth might get you fired
3        15 civilians killed in single us airstrike hav...
4        iranian woman jailed for fictional unpublished...
                               ...                        
18280    rapper t i   trump a  poster child for white s...
18281    n f l  playoffs  schedule  matchups and odds  ...
18282    macy s is said to receive takeover approach by...
18283    nato  russia to hold parallel exercises in bal...
18284                            what keeps the f 35 alive
Name: title, Length: 18285, dtype: object

In [25]:
messages.title = messages.title.apply(text_process)

In [26]:
messages.title

0        hous dem aid even see comey letter jason chaff...
1          flynn hillari clinton big woman campu breitbart
2                                     truth might get fire
3              15 civilian kill singl us airstrik identifi
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
18280            rapper trump poster child white supremaci
18281      n f l playoff schedul matchup odd new york time
18282    maci said receiv takeov approach hudson bay ne...
18283             nato russia hold parallel exercis balkan
18284                                       keep f 35 aliv
Name: title, Length: 18285, dtype: object

In [27]:
messages.title.describe()

count     18285
unique    17750
top            
freq         51
Name: title, dtype: object

In [28]:
messages.title.apply(len).max()

299

In [29]:
onehot_rep = [one_hot(words,vocab_size)for words in messages['title']]
onehot_rep

[[1876, 1533, 858, 987, 4024, 3818, 1488, 1831, 1750, 2739],
 [4337, 1326, 628, 2045, 3462, 1949, 1189],
 [1216, 2735, 3839, 3847],
 [789, 4250, 4738, 534, 2789, 4180, 2201],
 [1503, 3462, 3856, 2769, 2432, 1727, 3462, 1744, 2521, 3608],
 [3887,
  2779,
  3124,
  2048,
  3327,
  684,
  4953,
  688,
  3511,
  3165,
  4138,
  3447,
  3834,
  2509,
  1189],
 [325, 2325, 3771, 1332, 2939, 681, 435, 1901, 2844, 2557, 1373],
 [1011, 1473, 4372, 3743, 1040, 1648, 684, 1528, 2844, 2557, 1373],
 [2105, 3249, 4777, 2351, 1439, 4831, 874, 53, 684, 2099],
 [3936, 823, 4969, 2752, 1879, 2414, 1224, 3250],
 [3001, 4881, 3998, 1860, 3300, 3245, 2657, 3559, 701, 2057, 948],
 [2789, 3941, 4024, 4831, 684, 1040],
 [1390, 3853, 3005, 3379, 1373, 2887, 2187, 2156, 859],
 [2751, 216, 3181, 1890, 1521, 3253, 1257, 2844, 2557, 1373],
 [2966, 261, 1639, 4091, 1967, 2844, 2557, 1373],
 [2044, 236, 173, 4961, 4642, 2261, 2364, 4776, 655, 3225, 4195, 4503],
 [2155, 1801, 1326],
 [4439, 3464, 799, 136, 684, 1950,

In [31]:
# embedding representation
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1831 1750 2739]
 [   0    0    0 ... 3462 1949 1189]
 [   0    0    0 ... 2735 3839 3847]
 ...
 [   0    0    0 ... 2844 2557 1373]
 [   0    0    0 ...  827 4950 2480]
 [   0    0    0 ... 2226 4676 4950]]


In [33]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1876,
       1533,  858,  987, 4024, 3818, 1488, 1831, 1750, 2739], dtype=int32)

In [32]:
len(embedded_docs)

18285

In [34]:
#creating model
embedding_vector_features = 40
model = Sequential()

model.add(Embedding(vocab_size,embedding_vector_features,input_length = sent_length))

model.add(LSTM(100))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [36]:
len(embedded_docs),y.shape

(18285, (18285,))

In [37]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.3,random_state=42)

In [40]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 9s 33ms/step - loss: 0.4757 - accuracy: 0.7470 - val_loss: 0.1945 - val_accuracy: 0.9182
Epoch 2/10
200/200 [==============================] - 6s 30ms/step - loss: 0.1358 - accuracy: 0.9458 - val_loss: 0.1924 - val_accuracy: 0.9233
Epoch 3/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0847 - accuracy: 0.9706 - val_loss: 0.2352 - val_accuracy: 0.9189
Epoch 4/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0487 - accuracy: 0.9833 - val_loss: 0.2351 - val_accuracy: 0.9132
Epoch 5/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0255 - accuracy: 0.9935 - val_loss: 0.2988 - val_accuracy: 0.9132
Epoch 6/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0153 - accuracy: 0.9954 - val_loss: 0.4228 - val_accuracy: 0.9089
Epoch 7/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0072 - accuracy: 0.9982 - val_loss: 0.4025 - val_accuracy: 0.9078

In [41]:
y_pred = model.predict_classes(X_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test,y_pred)

array([[2798,  309],
       [ 171, 2208]])

In [44]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(y_test,y_pred)

0.9125045570543201